In [ ]:
from flask import Flask
from flask import request
import time
import json


app = Flask(__name__)
 
@app.route("/process", methods=['POST'])
def hello():
    data = request.get_json()
    print(data)
    name = data['name']
    return app.response_class(
        response=json.dumps({'nevem':name}),
        status=200,
        mimetype='application/json' )
 
if __name__ == "__main__":
    app.run(port= 4545)



In [ ]:
import sys
sys.path.append("..")
from om.BiAPI import Worker
from om.BiData import BinanceData
import pandas as pd
from flask import Flask
import time
import json

app = Flask(__name__)

@app.route('/', methods=['GET'])
def _adatLeker(**kwargs):
    # lekér
    last_update = B.get_time()
    
    current_price=B.get_current_tickers()
    
    d1m = B.get_window_ticker_deltas(1*60*1000, in_percentage=True)
    d3m = B.get_window_ticker_deltas(3*60*1000, in_percentage=True)
    d5m = B.get_window_ticker_deltas(5*60*1000, in_percentage=True)
    d15m = B.get_window_ticker_deltas(15*60*1000, in_percentage=True)
    
    means = B.get_mean()

        
    data_change = {'d1m':d1m,
            'd3m':d3m,
            'd5m':d5m, 
            'd15m':d15m}

    # objektum készítés
    data_to_send = {'last_update': last_update,
                    'prices' : current_price,
                    'data_change':data_change,
                    'means':means}
    # post -olod
    # flask
    return app.response_class(
        response=json.dumps(data_to_send),
        status=200,
        mimetype='application/json' )


if __name__ == '__main__':
    B = Worker(api_key='okkuE5o6cBw9UfkoYG0xL8ENTYAYla6m9wSek0dKpjAG6EzcQXlO1QVO0rcKyi3X',
           api_secret='Yf7TfoZ9MlxiCyDKfTafY6jvS8Kez14NYcMl2a5yS1jPeOchr1x9aleQUVX0RTj5',
           means=[1*1*60*1000, 7*15*60*1000, 25*15*60*1000, 99*15*60*1000])

    B.event_add('data', _adatLeker)
    B.init()
    app.run(host='0.0.0.0', debug=True)

